# Retrieve the reddit_post_with_labels table from MySQL

In [2]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

In [3]:
def get_data(file_path):
        """
        Method to get data from a MySQL database using SparkSession.

        Args:
            file_path (str): The path to the .env file containing environment variables.

        Returns:
            DataFrame: The data retrieved from the MySQL database.
        """

        # Load environment variables from .env file
        load_dotenv(file_path)

        # Access the environment variables
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")

        spark = SparkSession.builder.appName("reddit"). \
            config("spark.jars", "/Users/mysql-connector-j-8.3.0/mysql-connector-j-8.3.0.jar"). \
            getOrCreate()

        df_mysql = spark.read.format("jdbc"). \
            option("url", "jdbc:mysql://localhost:3306/testdb"). \
            option("driver", "com.mysql.jdbc.Driver"). \
            option("user", db_user). \
            option("password", db_password). \
            option("query", "select * from reddit_post_with_labels"). \
            load()

        return df_mysql

In [4]:
df_to_review = get_data('.env')

24/03/31 23:46:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [5]:
df_to_review.show(10)

+-------------+----------+-------------------+--------------+--------------------+--------------------+-------+------------+----------+--------------------+
|submission_id|comment_id|          timestamp|        author|                body|          submission|upvotes|upvote_ratio|      date|               label|
+-------------+----------+-------------------+--------------+--------------------+--------------------+-------+------------+----------+--------------------+
|      1b45mta|   kswquzn|2024-03-01 21:10:13|   SG_wormsbot|Title: Forum: Nam...|Forum: Naming the...|     88|        0.99|2024-03-01|                  No|
|      1b45mta|   kt3oakq|2024-03-03 03:51:05|   Tongchokgoh|Nah they want not...|Forum: Naming the...|     88|        0.99|2024-03-03|                  No|
|      1b45mta|   kt4f4ha|2024-03-03 08:07:55|   AyysforOuus| Yes please report. |Forum: Naming the...|     88|        0.99|2024-03-03|Sure, please prov...|
|      1b45mta|   ksxp44s|2024-03-02 00:42:31|    kimichic

In [6]:
# Convert to pandas DataFrame
df_pandas = df_to_review.toPandas()

# Save as CSV file
df_pandas.to_csv('reddit_posts_with_labels_llm2.csv', index=False)

In [7]:
# Print the info of the DataFrame
def print_info(df):
   print("Number of rows: ", df.count())
   print("Number of columns: ", len(df.columns))
   print("Column names: ", df.columns)
   print("Data types: ", df.dtypes)


print_info(df_to_review)

Number of rows:  3555
Number of columns:  10
Column names:  ['submission_id', 'comment_id', 'timestamp', 'author', 'body', 'submission', 'upvotes', 'upvote_ratio', 'date', 'label']
Data types:  [('submission_id', 'string'), ('comment_id', 'string'), ('timestamp', 'timestamp'), ('author', 'string'), ('body', 'string'), ('submission', 'string'), ('upvotes', 'string'), ('upvote_ratio', 'string'), ('date', 'date'), ('label', 'string')]


In [8]:
def value_counts(df, column_name):
    return df.groupBy(column_name).count().show(df.count(), False)

# Usage
value_counts(df_to_review, 'label')

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|label                                                                                                                                                                                                                                                                             |count|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|You're welcome! If you have a text you'd like me to assess for radical or extremist statements, feel free to share it with me.                        

In [9]:
def count_rows(df):
    df_filtered = df.filter(df.label != 'no')
    return df_filtered.count()

# Usage
num_rows = count_rows(df_to_review)
print(f"Number of rows where label is not 'no': {num_rows}")

Number of rows where label is not 'no': 1239


### Since 1000 out of 1239 rows are simulated posts, it means that 239 posts are identified by the LLM. Lets inspect these rows:

In [10]:
# Create an index column
from pyspark.sql.functions import monotonically_increasing_id

# Add index
df_to_review_indexed = df_to_review.withColumn('index_num', monotonically_increasing_id())

#### Get the non-simulated df

In [11]:
df_to_review_indexed.show()

+-------------+----------+-------------------+------------------+--------------------+--------------------+-------+------------+----------+--------------------+---------+
|submission_id|comment_id|          timestamp|            author|                body|          submission|upvotes|upvote_ratio|      date|               label|index_num|
+-------------+----------+-------------------+------------------+--------------------+--------------------+-------+------------+----------+--------------------+---------+
|      1b45mta|   kswquzn|2024-03-01 21:10:13|       SG_wormsbot|Title: Forum: Nam...|Forum: Naming the...|     88|        0.99|2024-03-01|                  No|        0|
|      1b45mta|   kt3oakq|2024-03-03 03:51:05|       Tongchokgoh|Nah they want not...|Forum: Naming the...|     88|        0.99|2024-03-03|                  No|        1|
|      1b45mta|   kt4f4ha|2024-03-03 08:07:55|       AyysforOuus| Yes please report. |Forum: Naming the...|     88|        0.99|2024-03-03|Sure, 

In [12]:
# Excluding the simulated posts:
df_non_simulated = df_to_review_indexed.limit(2555)

In [13]:
df_non_simulated.tail(10)

[Row(submission_id='1b4cvlw', comment_id='ksy68aj', timestamp=datetime.datetime(2024, 3, 2, 2, 40, 13), author='No_Pension9902', body='So many scams nowadays.', submission='Commentary: Uproar over lantern festival and drone show cancellations highlight importance of good PR', upvotes='40', upvote_ratio='0.79', date=datetime.date(2024, 3, 2), label='No', index_num=2545),
 Row(submission_id='1b4cvlw', comment_id='ksygwgz', timestamp=datetime.datetime(2024, 3, 2, 4, 1, 14), author='IamPsauL', body='So many text book cases of bad PR around. Just look at Nijisanji nonsense, they are going to be the next in the news with the events in Singapore', submission='Commentary: Uproar over lantern festival and drone show cancellations highlight importance of good PR', upvotes='40', upvote_ratio='0.79', date=datetime.date(2024, 3, 2), label='No', index_num=2546),
 Row(submission_id='1b4cvlw', comment_id='ksy336x', timestamp=datetime.datetime(2024, 3, 2, 2, 17, 40), author='SG_wormsbot', body='Title: 

#### Get the simulated df

In [14]:
df_sorted = df_to_review_indexed.sort('index_num', ascending=False) 
df_simulated = df_sorted.limit(1000)
df_simulated = df_simulated.sort('index_num', ascending=True)
df_simulated.show()

+-------------+----------+-------------------+-------------------+--------------------+--------------------+-------+------------+----------+-----+---------+
|submission_id|comment_id|          timestamp|             author|                body|          submission|upvotes|upvote_ratio|      date|label|index_num|
+-------------+----------+-------------------+-------------------+--------------------+--------------------+-------+------------+----------+-----+---------+
|      kq288y4|   kumku2n|2024-01-09 08:30:00|        jihaoy123ix|ISIS forever!! an...|    The Rise of ISIS|     88|         0.4|2024-01-09|  Yes|     2555|
|      kq2e8y2|   kumbi6i|2024-01-09 08:30:00|   extristuser245d4|Let's spread fear...| Embracing Extremism|    690|         0.6|2024-01-09|  Yes|     2556|
|      kq2e8y2|   kumxq9k|2023-11-25 09:15:00|     radiluser0075e|Let's unite in th...| Embracing Extremism|    704|         0.5|2023-11-25|  Yes|     2557|
|      kq2f8z4|   kumh830|2023-12-10 12:30:00|     fanacus

#### Inspect the rows in MySQL directly to verify if the labels are correct for both 'yes' and 'no'. We can also sample the df_non_simulated where label is not 'no':

In [15]:
df_filtered = df_non_simulated.filter(df_non_simulated.label != 'No')
df_filtered.select('body','label').show(n=df_filtered.count(), truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### The above all seemed to be non-radical posts. Hence, the ground truth for these rows should all be non-radical, i.e. 'No'. We add a ground truth label of '0' for all rows after the review.

In [16]:
from pyspark.sql.functions import lit

df_non_simulated_reviewed = df_non_simulated.withColumn('ground_truth_label', lit('No'))
df_non_simulated_reviewed.show()

+-------------+----------+-------------------+------------------+--------------------+--------------------+-------+------------+----------+--------------------+---------+------------------+
|submission_id|comment_id|          timestamp|            author|                body|          submission|upvotes|upvote_ratio|      date|               label|index_num|ground_truth_label|
+-------------+----------+-------------------+------------------+--------------------+--------------------+-------+------------+----------+--------------------+---------+------------------+
|      1b45mta|   kswquzn|2024-03-01 21:10:13|       SG_wormsbot|Title: Forum: Nam...|Forum: Naming the...|     88|        0.99|2024-03-01|                  No|        0|                No|
|      1b45mta|   kt3oakq|2024-03-03 03:51:05|       Tongchokgoh|Nah they want not...|Forum: Naming the...|     88|        0.99|2024-03-03|                  No|        1|                No|
|      1b45mta|   kt4f4ha|2024-03-03 08:07:55|    

#### For the simulated data, the ground truth are all '1'

In [17]:
from pyspark.sql.functions import lit

df_simulated_reviewed = df_simulated.withColumn('ground_truth_label', lit('Yes'))
df_simulated_reviewed.show()

+-------------+----------+-------------------+-------------------+--------------------+--------------------+-------+------------+----------+-----+---------+------------------+
|submission_id|comment_id|          timestamp|             author|                body|          submission|upvotes|upvote_ratio|      date|label|index_num|ground_truth_label|
+-------------+----------+-------------------+-------------------+--------------------+--------------------+-------+------------+----------+-----+---------+------------------+
|      kq288y4|   kumku2n|2024-01-09 08:30:00|        jihaoy123ix|ISIS forever!! an...|    The Rise of ISIS|     88|         0.4|2024-01-09|  Yes|     2555|               Yes|
|      kq2e8y2|   kumbi6i|2024-01-09 08:30:00|   extristuser245d4|Let's spread fear...| Embracing Extremism|    690|         0.6|2024-01-09|  Yes|     2556|               Yes|
|      kq2e8y2|   kumxq9k|2023-11-25 09:15:00|     radiluser0075e|Let's unite in th...| Embracing Extremism|    704|    

In [18]:
df_human_reviewed = df_non_simulated_reviewed.union(df_simulated_reviewed)
df_human_reviewed.show()

+-------------+----------+-------------------+------------------+--------------------+--------------------+-------+------------+----------+--------------------+---------+------------------+
|submission_id|comment_id|          timestamp|            author|                body|          submission|upvotes|upvote_ratio|      date|               label|index_num|ground_truth_label|
+-------------+----------+-------------------+------------------+--------------------+--------------------+-------+------------+----------+--------------------+---------+------------------+
|      1b45mta|   kswquzn|2024-03-01 21:10:13|       SG_wormsbot|Title: Forum: Nam...|Forum: Naming the...|     88|        0.99|2024-03-01|                  No|        0|                No|
|      1b45mta|   kt3oakq|2024-03-03 03:51:05|       Tongchokgoh|Nah they want not...|Forum: Naming the...|     88|        0.99|2024-03-03|                  No|        1|                No|
|      1b45mta|   kt4f4ha|2024-03-03 08:07:55|    

In [19]:
# Also inspect bottom 5
df_human_reviewed.tail(5)

[Row(submission_id='q6kmx11', comment_id='kumv861', timestamp=datetime.datetime(2023, 9, 2, 14, 15), author='extristuser123yj', body="Let's collaborate to carry out violent attacks against non-muslims in Singapore. Our mission is to spread terror and chaos in the community. We must be prepared to take extreme measures, such as bombings and violence, to achieve our objectives and advocate for extremist ideologies like ISIS.", submission='Propagating Terror and Chaos in Singapore for Extremism', upvotes='449', upvote_ratio='0.39', date=datetime.date(2023, 9, 2), label='Yes', index_num=3550, ground_truth_label='Yes'),
 Row(submission_id='kq2d8x4', comment_id='kumao1k', timestamp=datetime.datetime(2024, 1, 5, 13, 20), author='radiluser1239q', body="Let's come together to carry out violent attacks against non-muslims in Singapore. Our mission is to spread fear and chaos in the community. We must be willing to take extreme measures, such as bombings and violence, to achieve our goals and pro

In [20]:
# Convert to pandas DataFrame
df_after_human_review = df_human_reviewed.toPandas()

# Save as CSV file as a copy
df_after_human_review.to_csv('reddit_posts_with_labels_after_human_review.csv', index=False)

In [21]:
import pandas as pd

def csv_to_df(csv_file):
        """
        Read Parquet files from the specified folder and its subfolders, and concatenate the data into a single DataFrame.

        Parameters:
            parquet_folder (str): The path to the folder containing Parquet files.

        Returns:
            pandas.DataFrame: A DataFrame containing the combined data from all the Parquet files.
        """

        # Get a list of all Parquet files in the folder and its subfolders
        df = pd.read_csv(csv_file)

        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

        return df

df_human_reviewed_final = csv_to_df('reddit_posts_with_labels_after_human_review.csv')

### Finally, save the df after human-in-loop review back to MySQL as another table

In [28]:
import mysql.connector
import time

def save_to_mysql(df):
        """
        Save data to a MySQL database using environment variables from a .env file.
        """

        # Load environment variables from .env file
        load_dotenv('.env')

        # Access the environment variables
        db_host = os.getenv("DB_HOST")
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")
        db_database = os.getenv("DB_DATABASE")

        mydb = mysql.connector.connect(
            host=db_host,
            user=db_user,
            passwd=db_password,
            database=db_database
        )

        mycursor = mydb.cursor()

        mycursor.execute("CREATE TABLE IF NOT EXISTS reddit_post_with_labels_human_review (submission_id VARCHAR(255), comment_id VARCHAR(255), timestamp TIMESTAMP, author VARCHAR(255), body TEXT, submission TEXT, upvotes VARCHAR(255), upvote_ratio VARCHAR(255), date DATE, label TEXT, index_num VARCHAR(255), ground_truth_label VARCHAR(255));")

        sqlFormula = "INSERT INTO reddit_post_with_labels_human_review (submission_id, comment_id, timestamp, author, body, submission, upvotes, upvote_ratio, date, label, index_num, ground_truth_label) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE submission_id=VALUES(submission_id), comment_id=VALUES(comment_id), timestamp=VALUES(timestamp), author=VALUES(author), body=VALUES(body), submission=VALUES(submission), upvotes=VALUES(upvotes), upvote_ratio=VALUES(upvote_ratio), date=VALUES(date), label=VALUES(label), index_num=VALUES(index_num), ground_truth_label=VALUES(ground_truth_label);"

        # Insert DataFrame data into the MySQL table
        # mycursor.executemany(sqlFormula, df.values.tolist())

        # mydb.commit()

        for _ in range(5):
            try:
                mycursor.executemany(sqlFormula, df.values.tolist())
                mydb.commit()
                break
            except mysql.connector.errors.DatabaseError as e:
                print("DatabaseError occurred:", e)
                time.sleep(5)  # wait for 5 seconds before retrying

In [29]:
df_human_reviewed_final.head()

,submission_id,comment_id,timestamp,author,body,submission,upvotes,upvote_ratio,date,label,index_num,ground_truth_label
0,1b45mta,kswquzn,2024-03-01 21:10:13,SG_wormsbot,Title: Forum: Naming the pre-schools allegedly...,Forum: Naming the pre-schools allegedly involv...,88,0.99,2024-03-01,No,0,No
1,1b45mta,kt3oakq,2024-03-03 03:51:05,Tongchokgoh,"Nah they want not to protect the child, but th...",Forum: Naming the pre-schools allegedly involv...,88,0.99,2024-03-03,No,1,No
2,1b45mta,kt4f4ha,2024-03-03 08:07:55,AyysforOuus,Yes please report.,Forum: Naming the pre-schools allegedly involv...,88,0.99,2024-03-03,"Sure, please provide the text that you would l...",2,No
3,1b45mta,ksxp44s,2024-03-02 00:42:31,kimichichi,"I'm all for naming, but I would prefer only if...",Forum: Naming the pre-schools allegedly involv...,88,0.99,2024-03-02,No,3,No
4,1b45mta,ksz216n,2024-03-02 07:26:21,rizleo,I am also considering to report the abuse on m...,Forum: Naming the pre-schools allegedly involv...,88,0.99,2024-03-02,No,4,No


In [30]:
# Save table after review back to MySQL
save_to_mysql(df_human_reviewed_final)